In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = "/content/drive/MyDrive/DS340_Final_Project/speech_augmented_spectrograms_noise/noise_0.02"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image dimensions
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 1155 images belonging to 8 classes.
Found 285 images belonging to 8 classes.


In [ ]:
from tensorflow.keras import layers
from tensorflow import keras
input_shape = (128, 128, 3)

In [ ]:
model3 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5), # add dropout
        layers.Dense(128, activation='relu'),
        layers.Dense(8, activation='softmax'),
    ]
)

In [ ]:
model3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

import keras.callbacks
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)  # Quit after 3 rounds of no validation loss improvement
]

model3.fit(train_generator, epochs=15, validation_data=val_generator, callbacks=callbacks) # add early stopping

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


In [ ]:
model5 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)), # add max pooling
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu'), # add another layer
        layers.MaxPooling2D(pool_size=(2, 2)), # add max pooling
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dense(8, activation='softmax'),
    ]
)

In [ ]:
model5.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

import keras.callbacks
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)  # Quit after 3 rounds of no validation loss improvement
]

model5.fit(train_generator, epochs=15, validation_data=val_generator, callbacks=callbacks)

In [ ]:
from keras import regularizers
from keras import layers, regularizers, Sequential

model6 = Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)), # add l2 regularization
        # remove max pooling
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)), #add l2 regularization
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)), #add l2 regularization
        # remove max pooling
        layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)), #add l2 regularization
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dropout(0.6), # change dropout from 0.5 to 0.6
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4)), #add l2 regularization
        layers.Dense(8, activation='softmax'),
    ]
)

In [ ]:
model6.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"]) # modify learning rate

import keras.callbacks
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)  # Quit after 3 rounds of no validation loss improvement
]

model6.fit(train_generator, epochs=15, validation_data=val_generator, callbacks=callbacks)